
# InstaCart Data
The InstaCart Data (https://www.kaggle.com/c/instacart-market-basket-analysis/data) is slightly too large for in-memory merging, so let's use a sqlite3 database to store and do large transformations. 

Load each of the tables into the database

In [ ]:
# Create database to hold data
import csv, sqlite3

## orders.csv


In [48]:
# Connect to database
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create orders table in sqlite3
cur.execute("DROP TABLE IF EXISTS orders;")
cur.execute("CREATE TABLE orders (order_id integer, user_id integer, eval_set string, order_number integer, order_dow integer,order_hour_of_day integer,days_since_prior_order integer);")

with open('data/orders.csv') as csvfile: 
    reader = csv.DictReader(csvfile)
    for row in reader:
        cur.execute("INSERT INTO orders (order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order ) VALUES (?, ?, ?, ?, ?, ?, ?);", list(row.values()))

con.commit()
con.close()

## order_products__prior.csv

In [68]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create prior order producs table in sqlite3
cur.execute("DROP TABLE IF EXISTS products_prior;")
cur.execute("CREATE TABLE products_prior (order_id integer, product_id integer, add_to_cart_order integer, reordered integer);")

with open('data/order_products__prior.csv', encoding='utf-8', errors='ignore') as csvfile: 
    reader = csv.DictReader(csvfile)
    for row in reader:
        cur.execute("INSERT INTO products_prior (order_id,product_id,add_to_cart_order,reordered) VALUES (?, ?, ?, ?);", list(row.values()))

con.commit()
con.close()

## products.csv

In [61]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create prior order producs table in sqlite3
cur.execute("DROP TABLE IF EXISTS products;")
cur.execute("CREATE TABLE products (product_id integer, product_name string, aisle_id integer, department_id integer);")

with open('data/products.csv', encoding='utf-8', errors='ignore') as csvfile: 
    reader = csv.DictReader(csvfile)
    for row in reader:
        cur.execute("INSERT INTO products (product_id,product_name,aisle_id,department_id) VALUES (?, ?, ?, ?);", list(row.values()))

con.commit()
con.close()

## aisles.csv

In [64]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create prior order producs table in sqlite3
cur.execute("DROP TABLE IF EXISTS aisles;")
cur.execute("CREATE TABLE aisles (aisle_id integer, aisle string);")

with open('data/aisles.csv', encoding='utf-8', errors='ignore') as csvfile: 
    reader = csv.DictReader(csvfile)
    for row in reader:
        cur.execute("INSERT INTO aisles (aisle_id,aisle) VALUES (?, ?);", list(row.values()))

con.commit()
con.close()

## departments.csv

In [65]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create prior order producs table in sqlite3
cur.execute("DROP TABLE IF EXISTS departments;")
cur.execute("CREATE TABLE departments (department_id integer, department string);")

with open('data/departments.csv', encoding='utf-8', errors='ignore') as csvfile: 
    reader = csv.DictReader(csvfile)
    for row in reader:
        cur.execute("INSERT INTO departments (department_id,department) VALUES (?, ?);", list(row.values()))

con.commit()
con.close()

## order_products__train.csv

In [67]:
con = sqlite3.connect("instacart.db")
cur = con.cursor()

# Create prior order producs table in sqlite3
cur.execute("DROP TABLE IF EXISTS products_train;")
cur.execute("CREATE TABLE products_train (order_id integer, product_id integer, add_to_cart_order integer, reordered integer);")

with open('data/order_products__train.csv', encoding='utf-8', errors='ignore') as csvfile: 
    reader = csv.DictReader(csvfile)
    for row in reader:
        cur.execute("INSERT INTO products_train (order_id,product_id,add_to_cart_order,reordered) VALUES (?, ?, ?, ?);", list(row.values()))

con.commit()
con.close()

# Other Prep

In [192]:
# Import stuff
import numpy as np
import pandas as pd
import time

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import zipfile
import csv
import glob
import os
import io

import unittest


%matplotlib inline

Set style

In [69]:
sns.set(style="whitegrid", font_scale=1.3)
matplotlib.rcParams["legend.framealpha"] = 1
matplotlib.rcParams["legend.frameon"] = True

Just for the sake of reproducibility

In [71]:
np.random.seed(21)

## Split testset into train and validation

The orders table consists of three types of orders: 

| eval_set  | Records | Purpose |
|:----------|:-----------|:-----|
|prior |3,214,874 |Prior orders that provide history for other two sets  |
|train |131,209 |The last order for training on
|test | 75,000 |Orders for which to predict the future orders then submit on Kaggle.com

Thus, test is off limits and not really touched. But from train, I can make a validation set by splitting it up. Let's do it with the last digit of order_id. 

**If order_id % 10 >= 7 and eval_set = 'train', then 'validation'**

## Create dataframe with actual results

Go to the answers and make a dataframe that matches the sample submission. From the Kaggle evaluation page: 

> For each order_id in the test set, you should predict a space-delimited list of product_ids for that order. If you wish to predict an empty order, you should submit an explicit 'None' value. You may combine 'None' with product_ids. The spelling of 'None' is case sensitive in the scoring metric. The file should have a header and look like the following:

>```
order_id,products  
17,1 2  
34,None  
137,1 2 3  
etc.
```

It is easier to compare if testing is a dict object, keys being the order id, value being the set of products

In [178]:
conn = sqlite3.connect("instacart.db")
cur = conn.cursor()

# Query sqlite3 for the validation orders and their products
actual_results_raw = pd.read_sql_query("SELECT A.order_id as order_id, "
                                   "  COALESCE(CAST(B.product_id as text), 'None') as product_id "
                                   "FROM orders A LEFT JOIN ( "
                                   "  SELECT order_id, product_id "
                                   "  FROM products_train "
                                   "  WHERE order_id % 10 >= 7 "
                                   "    AND reordered = 1 ) B "
                                   "  ON A.order_id = B.order_id "
                                   "WHERE A.eval_set = 'train' "
                                   "  AND A.order_id % 10 >= 7;" , conn)

con.close()

In [184]:
actual_results = {}

# Convert evaluation framework: results[order_id] = set([product1, product2, ...])
for row in actual_results_raw.itertuples():
    if row.order_id not in actual_results:
        actual_results[row.order_id] = set()
    actual_results[row.order_id].add(row.product_id)


In [190]:
# Look at a few actual results:
for i in range(5):
    rando = np.random.randint(0,actual_results_raw.shape[0])
    print(str(actual_results_raw.iloc[rando].order_id) + ": " 
          + str(actual_results[actual_results_raw.iloc[rando].order_id]) )

421779: {'48745', '24957', '45007', '48094', '8146', '17920', '43789', '32710', '2979', '23236', '7503'}
1159799: {'16262', '5692', '19057', '20693', '27845', '27864', '44292', '13541', '47630', '26317', '30169', '14947', '33787', '3957', '13176', '11925', '27966', '8021'}
1000247: {'3381', '6825', '20930', '13328', '39982'}
848619: {'31371', '29871', '27548', '38689', '6631', '46107', '35221', '44560', '46584', '11440', '4367', '30720'}
254328: {'3706', '4724', '16797', '8571', '8803', '12341'}


In [186]:
# Look for null: 
actual_results_none = actual_results_raw[actual_results_raw.product_id == "None"]
actual_results_none.head()

,order_id,product_id
64,2436259,None
275,2887658,None
353,1976038,None
412,394279,None
604,2724968,None


## Create dummy comparison set

Let's predict 'None' for every single order, just to practice calculating F1

In [293]:
# dictionary of orders where the products predicted will all be none
dummy_results = {}

# Convert evaluation framework: results[order_id] = set([product1, product2, ...])
for row in actual_results_raw.itertuples():
    if row.order_id not in dummy_results:
        dummy_results[row.order_id] = set()
    dummy_results[row.order_id].add('None')
    
# Look at a few actual results:
for i in range(5):
    rando = np.random.randint(0,actual_results_raw.shape[0])
    print(str(actual_results_raw.iloc[rando].order_id) + ": " 
          + str(dummy_results[actual_results_raw.iloc[rando].order_id]) )


3187808: {'None'}
1436219: {'None'}
1208039: {'None'}
152837: {'None'}
1780247: {'None'}


## Create F1 score calculator

The F1 mean score is used in this competition. 

```
F1 = 2 * (Precision * Recall) / (Precision + Recall)
```

...where...

```
Precision = True Positives / (True Positives + False Positives)
Recall = True Positives / (True Positives + False Negatives)
```

In [294]:
# F1 takes two dictionaries, predicted and actual, and compares the two, returning the F1 score, 
# ...or raising an error
def f1(pred,actual):
    # Error Checking input:
    # Are they the same size?
    if len(pred.keys()) != len(actual.keys()):
        raise AssertionError('Prediction set is not the same size as actual set')
    
    # Create aggregate scores:
    true_positives, false_positives, false_negatives = 0, 0, 0
    
    # For each order
    for order in actual.keys():
        if order not in pred:
            raise KeyError('order in actual not in prediction')
        else:
            # True positives is the size of the intersection
            true_positives += len(actual[order].intersection(pred[order]))
            # False positives are products in the prediction not in actual
            false_positives += len(pred[order] - actual[order])
            # False negatives are products in actual not in prediction
            false_negatives += len(actual[order] - pred[order])
    
    # Calculate F1
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    F1 = 2 * ( (precision * recall) / (precision + recall) )
    
    print("True Positives:  " + str(true_positives))
    print("False Positives: " + str(false_positives))
    print("False Negatives: " + str(false_negatives))
    print("Precision:       " + str(precision))
    print("Recall:          " + str(recall))
    print("----------------------------")
    print("F1: " + str(F1))
          
    return F1, true_positives, false_positives, false_negatives
    

In [295]:
dummy_f1, dummy_true, dummy_falsep, dummy_falsen = f1(dummy_results, actual_results)

True Positives:  2586
False Positives: 37002
False Negatives: 251536
Precision:       0.0653228250985147
Recall:          0.010176214574102203
----------------------------
F1: 0.017609206360014982


## Track Results

Per Jimmy, 

> Also one should always track the results of the experiments to be able to compare different approaches. Let's create pandas DataFrame for this purpose. 

I couldn't say it better. 

In [304]:
results = pd.DataFrame(columns=["Model", "F1","True Positives", "False Positive", "False Negatives"])
results

,Model,F1,True Positives,False Positive,False Negatives


In [307]:
# Append the dummy results:
dummy_results = pd.DataFrame([['Dummy - None', dummy_f1, dummy_true, dummy_falsep, dummy_falsen]]
                             , columns=["Model", "F1","True Positives", "False Positive", "False Negatives"])
results = results.append(dummy_results)
results

,Model,F1,True Positives,False Positive,False Negatives
0,Dummy - None,0.017609,2586.0,37002.0,251536.0


# Query and Review Data

Let's look at the data and do some quick assertions.

In [50]:
# Check data
con = sqlite3.connect("instacart.db")
cur = con.cursor()

cur.execute('SELECT * FROM products_prior LIMIT 5;')
all_rows = cur.fetchall()
print(all_rows)

con.commit()
con.close()

[(2, 33120, 1, 1), (2, 28985, 2, 1), (2, 9327, 3, 0), (2, 45918, 4, 1), (2, 30035, 5, 0)]


In [54]:
# Check that there are 5 products purchased 99 times
con = sqlite3.connect("instacart.db")
cur = con.cursor()

cur.execute('SELECT * FROM user_prod_freq where purch_freq = 99;')
all_rows = cur.fetchall()
print(all_rows)

con.commit()
con.close()

[(17997, 4210, 99), (41356, 6583, 99), (41356, 14366, 99), (41356, 38652, 99), (141736, 25133, 99)]


# Naive Reorder Logic

Let us set a really simple baseline - for each user, find the number of reordered items in a basket, call it n, then take their n most frequently ordered items. 

Take the prior, then predict on validation and calculate accuracy

## Average Reorder Per User

In [77]:
# For each user, calculate average number of reordered items in a basket
conn = sqlite3.connect("instacart.db")
cur = conn.cursor()

# Number of reorders per order
cur.execute("CREATE TABLE reorder_per_order AS "
            "SELECT order_id, SUM(reordered) as reordered "
            "FROM products_prior "
            "GROUP BY order_id;")

conn.commit()
conn.close()

In [86]:
# Join to orders to calculate average per user
conn = sqlite3.connect("instacart.db")
cur = conn.cursor()

# Drop prior table if exists
cur.execute("DROP TABLE IF EXISTS reorder_per_user_prior;")

# Average reorders per user_id
cur.execute("CREATE TABLE reorder_per_user_prior AS "
            "SELECT A.user_id as user_id, AVG(B.reordered) as reorder_avg "
            "FROM orders A INNER JOIN reorder_per_order B "
            "ON A.order_id = B.order_id "
            "WHERE A.order_number > 1 " # Reorders arent possible on first order
            "GROUP BY A.user_id;")

conn.commit()
conn.close()

In [87]:
# Spit out table into a dataframe
conn = sqlite3.connect("instacart.db")
cur = conn.cursor()

reorder_per_user_prior = pd.read_sql_query("SELECT * FROM reorder_per_user_prior;", conn)

# verify that result of SQL query is stored in the dataframe
print(reorder_per_user_prior.head())

conn.close()


   user_id  reorder_avg
0        1     4.555556
1        2     7.153846
2        3     5.000000
3        4     0.250000
4        5     4.666667


## Most Reordered Products Per User

In [225]:
# Join products to orders, count reorders
conn = sqlite3.connect("instacart.db")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS reorder_by_user_product;")

# Calculate reorder_count per user per product
cur.execute("CREATE TABLE reorder_by_user_product AS "
            "SELECT A.user_id as user_id, "
            "  B.product_id as product_id, "
            "  count(*) as reorder_count "
            "FROM orders A INNER JOIN products_prior B "
            "  ON A.order_id = B.order_id "
            "WHERE B.reordered = 1 "
            "GROUP BY A.user_id, B.product_id;")

# Dump to python
reordered_products_per_user = pd.read_sql_query("SELECT * FROM reorder_by_user_product;", conn)

conn.commit()
conn.close()

In [92]:
# Let's look at it
reordered_products_per_user.head()

,user_id,product_id,reorder_count
0,1,196,9
1,1,10258,8
2,1,12427,9
3,1,13032,2
4,1,13176,1


In [226]:
# Rank products by user
reordered_products_per_user["reorder_rank"] = reordered_products_per_user.groupby('user_id')['reorder_count'].rank(method="first", ascending=False)

In [227]:
reordered_products_per_user.head(25)

,user_id,product_id,reorder_count,reorder_rank
0,1,196,9,1.0
1,1,10258,8,3.0
2,1,12427,9,2.0
3,1,13032,2,5.0
4,1,13176,1,7.0
5,1,25133,7,4.0
6,1,26088,1,8.0
7,1,26405,1,9.0
8,1,46149,2,6.0
9,1,49235,1,10.0


## Predict for Validation

In [245]:
conn = sqlite3.connect("instacart.db")
cur = conn.cursor()

# Grab validation orders and user ids
validation_4_pred = pd.read_sql_query("SELECT order_id, user_id "
                                      "FROM orders "
                                      "WHERE eval_set = 'train' "
                                      "  AND order_id % 10 >= 7;", conn)

conn.close()

In [272]:
# Convert to dictionaries to speed everything up

# Order ID: user_id
validation_4_pred_dict = validation_4_pred.set_index('order_id')['user_id'].to_dict()

# user_id: reorder_avg
reorder_per_user_prior_dict = reorder_per_user_prior.set_index('user_id')['reorder_avg'].to_dict()

In [289]:
# Create a dictionary to house predictions
naive_pred = {}

progress_counter = 0 

# For each order:
for order in validation_4_pred_dict.keys():
    
    # Watch the progress
    progress_counter += 1
    if progress_counter % 1000 == 0:
        print(progress_counter/39588)
        
    # look up the reorder_count by user_id in reorder_per_user_prior
    reorder_count = np.floor(reorder_per_user_prior_dict[validation_4_pred_dict[order]])
    
    # Create an entry in naive_pred:
    naive_pred[order] = set()
      
    # If the reorder count is 0, add "None"
    if reorder_count == 0:
        naive_pred[order].add("None")
    else:
        # Otherwise, add most frequently reordered items:

        # Get the user_id's products
        reorder_products = reordered_products_per_user[reordered_products_per_user.user_id == validation_4_pred_dict[order]]

        # Convert products to dictionary by rank
        reorder_prods_dict = reorder_products.set_index('reorder_rank')['product_id'].to_dict()
      
        # For each rank less than avg:
        for itemNum in range(int(reorder_count)):
            # Add it to the prediction
            naive_pred[order].add(reorder_prods_dict[itemNum+1])


0.02526017985248055
0.0505203597049611
0.07578053955744164
0.1010407194099222
0.12630089926240276
0.15156107911488328
0.17682125896736384
0.2020814388198444
0.22734161867232494
0.2526017985248055
0.27786197837728605
0.30312215822976657
0.32838233808224715
0.3536425179347277
0.37890269778720825
0.4041628776396888
0.42942305749216936
0.4546832373446499
0.47994341719713046
0.505203597049611
0.5304637769020916
0.5557239567545721
0.5809841366070526
0.6062443164595331
0.6315044963120138
0.6567646761644943
0.6820248560169748
0.7072850358694553
0.732545215721936
0.7578053955744165
0.783065575426897
0.8083257552793776
0.8335859351318582
0.8588461149843387
0.8841062948368192
0.9093664746892998
0.9346266545417803
0.9598868343942609
0.9851470142467414


In [288]:
len(validation_4_pred_dict.keys())
#reorder_prods_dict[4]

39588

In [298]:
naive_f1, naive_true, naive_falsep, naive_falsen = f1(naive_pred, actual_results)

True Positives:  887
False Positives: 183617
False Negatives: 253235
Precision:       0.0048074838485886486
Recall:          0.003490449469152612
----------------------------
F1: 0.004044447889545991


In [308]:
# Append the naive results:
naive_results = pd.DataFrame([['Naive - predict most reordered', naive_f1, naive_true, naive_falsep, naive_falsen]]
                             , columns=["Model", "F1","True Positives", "False Positive", "False Negatives"])
results = results.append(naive_results)
results

,Model,F1,True Positives,False Positive,False Negatives
0,Dummy - None,0.017609,2586.0,37002.0,251536.0
0,Naive - predict most reordered,0.004044,887.0,183617.0,253235.0
